In [0]:
import numpy as np
import pyspark as ps
import pyspark.pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

#Configuration & Optimisation de l'engin Spark
conf = SparkConf()
#conf.set('spark.executor.memory', '2g')
#conf.set('spark.driver.memory', '2g')

#Pandas configurations
#pd.options.display.max_rows = 5

spark = (
    SparkSession.builder
        .master("local")
        .appName("TransformationExample")
        .config("spark.sql.ansi.enabled", "false")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .getOrCreate()
)

#Chargement
df = pd.read_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/cf_asset_mort_init.csv", sep=";")

In [0]:
    def add_cashflow_columns(df, num_cashflows=1200):
        colones = dict()
        for i in range(1200):
            colones[f'cash_flow_{i+1}'] = 0
        return df.reindex(columns=list(df.columns) + list(colones.keys()), fill_value=0.0)

    def compute_notional_cul(versement, tx_interet, maturity, solde_capital):
        notional_cul = np.zeros(maturity + 1,dtype=float)
        notional_cul[0] = solde_capital

        for i in range(maturity):
            notional_cul[i+1] = notional_cul[i] - versement + notional_cul[i] * tx_interet 
        return notional_cul

    def compute_cashflows(serie):
        maturity = serie['maturity']
        tx_interet = serie["taux_hyp"] / serie["frequence"]
        notional_cul = compute_notional_cul(serie['versement'], tx_interet, maturity, serie['solde_capital'])

        for i in range(maturity):
            serie[f'cash_flow_{i+1}'] = serie['versement']
            serie[f'cash_flow_{maturity}'] += notional_cul[maturity]     
        return serie

    def projection_cf_mortgage_opt(data):
        df = add_cashflow_columns(data)
        return df.apply(compute_cashflows, axis='columns')

In [0]:
#25-06-30
def projection_cf_mortgage(data):

    # Initialize the DataFrame
    cfs = np.zeros((len(data), 1200))

    # Iterate over each row in the data
    for j in range(len(data)):
        if data.iloc[j]['maturity']>0:
            tx_interet = data.iloc[j]['taux_hyp'] / data.iloc[j]['frequence']
            maturity = data.iloc[j]['maturity']
            pay_cap = data.iloc[j]['versement']
            notionnel_cul = np.zeros(maturity + 1)
            notionnel_cul[0] = data.iloc[j]['solde_capital']

            for k in range(maturity):
                notionnel_cul[k + 1] = notionnel_cul[k] - pay_cap + notionnel_cul[k]*tx_interet
                cfs[j, k] += pay_cap

            cfs[j, maturity - 1] += notionnel_cul[maturity]

    # Convert the NumPy array to a DataFrame
    cfs_df = pd.DataFrame(cfs, columns=[f'Cash Flow {i}' for i in range(1, cfs.shape[1] + 1)])

    # Concatenate the data with the cash flows
    result = pd.concat([data.reset_index(drop=True), cfs_df], axis=1)
    return result

In [0]:
result = projection_cf_mortgage(df)
display(result)